<a href="https://colab.research.google.com/github/Codes-of-Hermit/Finance_codes/blob/main/FICO_score_Rating_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

# 1. Load and Preprocess Data
from google.colab import files
uploaded = files.upload()
def load_and_aggregate_data(filename='Task 3 and 4_Loan_Data.csv'):
    df = pd.read_csv(filename)

    # We only need FICO score and Default status
    data = df[['fico_score', 'default']].copy()

    # Sort by FICO score to prepare for sequential bucketing
    data = data.sort_values(by='fico_score')

    # Aggregate data by FICO score to speed up calculations
    # We want: unique_fico | total_people | total_defaults
    agg_data = data.groupby('fico_score')['default'].agg(['count', 'sum']).reset_index()
    agg_data.columns = ['fico_score', 'total', 'defaults']

    return agg_data

# 2. Log-Likelihood Calculation
def calculate_log_likelihood(n, k):
    """
    Calculate LL for a single bucket.
    n: total observations in bucket
    k: defaults in bucket
    """
    p = k / n if n > 0 else 0
    if p == 0 or p == 1:
        return 0  # Limit of p*ln(p) as p->0 is 0
    return k * np.log(p) + (n - k) * np.log(1 - p)

# 3. Dynamic Programming to Find Optimal Buckets
def get_optimal_boundaries(df, num_buckets):
    """
    Uses Dynamic Programming to maximize Log-Likelihood.
    Returns the cutoff indices for the sorted dataframe.
    """
    n_scores = len(df)

    # Precompute cumulative sums for O(1) range queries
    cum_total = np.cumsum(df['total'].values)
    cum_defaults = np.cumsum(df['defaults'].values)

    # Helper to get sum in range [i, j] (inclusive)
    def get_stats(i, j):
        total = cum_total[j] - (cum_total[i-1] if i > 0 else 0)
        defaults = cum_defaults[j] - (cum_defaults[i-1] if i > 0 else 0)
        return total, defaults

    # dp[k][i] stores the max LL using k buckets for the first i+1 scores
    dp = np.full((num_buckets + 1, n_scores), -np.inf)
    # boundaries[k][i] stores the split point for reconstruction
    boundaries = np.zeros((num_buckets + 1, n_scores), dtype=int)

    # Initialize first bucket (k=1)
    for i in range(n_scores):
        t, d = get_stats(0, i)
        dp[1][i] = calculate_log_likelihood(t, d)

    # Fill DP table for k = 2 to num_buckets
    for k in range(2, num_buckets + 1):
        for i in range(k - 1, n_scores): # Need at least k items for k buckets
            # Iterate over possible split points j
            # We split into [0...j] (optimized previously) and [j+1...i] (new bucket)
            # Optimization: FICO scores are dense, we can limit search if needed,
            # but N~550 is small enough for full search.
            for j in range(k - 2, i):
                t, d = get_stats(j + 1, i)
                current_ll = calculate_log_likelihood(t, d)

                if dp[k-1][j] + current_ll > dp[k][i]:
                    dp[k][i] = dp[k-1][j] + current_ll
                    boundaries[k][i] = j

    # Backtrack to find the actual boundaries
    final_boundaries = []
    curr_idx = n_scores - 1
    for k in range(num_buckets, 1, -1):
        split_idx = boundaries[k][curr_idx]
        final_boundaries.append(df.iloc[split_idx]['fico_score'])
        curr_idx = split_idx

    final_boundaries.reverse()
    return final_boundaries

# --- Execution ---

# Load Data
df_agg = load_and_aggregate_data()

# Define desired number of buckets (ratings)
num_buckets = 10

# Get Boundaries
boundaries = get_optimal_boundaries(df_agg, num_buckets)

print(f"Optimal FICO Score Boundaries for {num_buckets} buckets:")
print(boundaries)

# Verify Output
print("\nBucket Ranges:")
start = 300 # FICO min
for b in boundaries:
    print(f"{start} - {b}")
    start = b + 1
print(f"{start} - 850") # FICO max

Saving Task 3 and 4_Loan_Data.csv to Task 3 and 4_Loan_Data (1).csv
Optimal FICO Score Boundaries for 10 buckets:
[np.int64(520), np.int64(552), np.int64(580), np.int64(611), np.int64(649), np.int64(696), np.int64(732), np.int64(752), np.int64(753)]

Bucket Ranges:
300 - 520
521 - 552
553 - 580
581 - 611
612 - 649
650 - 696
697 - 732
733 - 752
753 - 753
754 - 850
